In [72]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score
%matplotlib inline

In [73]:
df = pd.read_csv('https://raw.githubusercontent.com/aczepielik/KRKtram/master/reports/report_07-23.csv')
df.head()

,index,time_stamp,stop,stopName,number,direction,plannedTime,vehicleId,tripId,status,delay,seq_num
0,1,2018-07-23 06:00:45,378,Os.Piastów,21,Kopiec Wandy,2018-07-23 05:59:00,NaN,6351558574044883205,PLANNED,1,1.0
1,1,2018-07-23 06:00:47,612,Borsucza,22,Walcownia,2018-07-23 06:00:00,6.352185e+18,6351558574044899587,STOPPING,0,7.0
2,1,2018-07-23 06:00:48,572,Smolki,11,Czerwone Maki P+R,2018-07-23 06:00:00,6.352185e+18,6351558574044670211,STOPPING,0,10.0
3,1,2018-07-23 06:00:49,319,Jubilat,1,Wzgórza K.,2018-07-23 05:59:00,NaN,6351558574044363010,PLANNED,1,3.0
4,1,2018-07-23 06:00:49,322,Filharmonia,8,Bronowice Małe,2018-07-23 06:01:00,6.352185e+18,6351558574044592386,STOPPING,0,15.0


In [74]:
df['time_stamp'] = pd.to_datetime(df['time_stamp'])
df['plannedTime'] = pd.to_datetime(df['plannedTime'])
df['delay_in_seconds'] = df['time_stamp']-df['plannedTime']
df['delay_in_seconds'] = df['delay_in_seconds'].dt.total_seconds()
df.head()

,index,time_stamp,stop,stopName,number,direction,plannedTime,vehicleId,tripId,status,delay,seq_num,delay_in_seconds
0,1,2018-07-23 06:00:45,378,Os.Piastów,21,Kopiec Wandy,2018-07-23 05:59:00,NaN,6351558574044883205,PLANNED,1,1.0,105.0
1,1,2018-07-23 06:00:47,612,Borsucza,22,Walcownia,2018-07-23 06:00:00,6.352185e+18,6351558574044899587,STOPPING,0,7.0,47.0
2,1,2018-07-23 06:00:48,572,Smolki,11,Czerwone Maki P+R,2018-07-23 06:00:00,6.352185e+18,6351558574044670211,STOPPING,0,10.0,48.0
3,1,2018-07-23 06:00:49,319,Jubilat,1,Wzgórza K.,2018-07-23 05:59:00,NaN,6351558574044363010,PLANNED,1,3.0,109.0
4,1,2018-07-23 06:00:49,322,Filharmonia,8,Bronowice Małe,2018-07-23 06:01:00,6.352185e+18,6351558574044592386,STOPPING,0,15.0,-11.0


In [75]:
df['seq_num'].fillna(-1 , inplace=True)
df['stop'] = df['stop'].factorize()[0]
df['hour'] = df['plannedTime'].dt.hour
df['number_direction'] = df.apply(lambda x: '{} {}'.format(x['number'] , x['direction']),axis = 1).factorize()[0]

In [76]:
feats = ['seq_num' , 'stop' , 'hour' , 'number_direction']
X = df[feats].values
Y = df['delay_in_seconds'].values
model = DecisionTreeRegressor(max_depth=10)
score = cross_val_score(model , X , Y , cv=3 , scoring='neg_mean_absolute_error')
np.mean(score) , np.std(score)

(-60.653375684509115, 2.037034514414553)